In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /MyDrive/knee-project

All of the model creation and training related work was done on the google colab. 

In [ ]:
%tensorflow_version 2.x

In [ ]:
import tensorflow
print(tensorflow.__version__)

In [ ]:
import cv2 as cv
import os
data_path = '/content/drive/MyDrive/knee-project'
categories = os.listdir(data_path)
labels = [i for i in range(len(categories))]

label_dict=dict(zip(categories,labels))
print(label_dict)
print(categories)
print(labels)

In [ ]:
img_size = 256
data = []
label = []

for category in categories:
  folder_path=os.path.join(data_path,category)
  img_names = os.listdir (folder_path)

  for img_name in img_names:
    img_path = os.path.join(folder_path, img_name)
    img = cv.imread(img_path)

    try:
      gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
      resized = cv.resize(gray,(img_size,img_size))
      data.append(resized)
      label.append(label_dict[category])
    except Exception as e:
      print ('Execption',e)

In [ ]:
import numpy as np
data = np.array(data)/255.0
data = np.reshape(data,(data.shape[0],img_size,img_size,1))
label = np.array(label)
from tensorflow.keras.utils import to_categorical
new_label=to_categorical(label)
print (new_label)

In [ ]:
new_label.shape

In [ ]:
data.shape

In [ ]:
data.shape[1:]

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

model = Sequential()

model.add(Conv2D(128,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
# first cnn layer followed by relu and maxpooling layers

model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#second convolution layer

model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#Third convolution layer

model.add(Flatten())
#flatten layer

model.add(Dense(128,activation='relu'))
#dense layer of 128 nuerons

model.add(Dropout(0,1))
model.add(Dense(64,activation='relu'))
#Dense layer of 64 nuerons

model.add(Dense(5,activation='softmax'))
#final layer with two outputs for two categories

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(data,new_label,test_size=0.1)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
for i in range (20):
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(np.squeeze(x_test[i]))
  plt.xlabel(categories[np.argmax(y_test[i])])

plt.show()

In [ ]:
history = model.fit(x_train, y_train, epochs=100, validation_split=0.2)

In [ ]:
model.save('model.h5')